In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 0. Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Preparação dos Dados

## 1.1 Trazer todas as informações para um único dataset

Leitura dos datasets

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/test.csv')
oil = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/oil.csv')
holidays_events = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/holidays_events.csv')
stores = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/stores.csv')
transactions = pd.read_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/transactions.csv')

In [ ]:
vars_train_001 = train.columns.to_list()
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,0
1,650431,2014-01-02,1,BABY CARE,0.0,0
2,650432,2014-01-02,1,BEAUTY,6.0,0
3,650433,2014-01-02,1,BEVERAGES,2524.0,0
4,650434,2014-01-02,1,BOOKS,0.0,0


In [ ]:
oil = oil.rename(columns={'price':'oil_price'})
oil.head()

,date,oil_price
0,2014-01-02,95.14
1,2014-01-03,93.66
2,2014-01-06,93.12
3,2014-01-07,93.31
4,2014-01-08,91.90


In [ ]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
1,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
2,2012-12-06,Holiday,Local,Quito,Fundacion de Quito,False
3,2013-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
4,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False


In [ ]:
# Renomear variável type
holidays_events = holidays_events.rename(columns={'type':'type_holiday'})
holidays_events.head()

,date,type_holiday,locale,locale_name,description,transferred
0,2012-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
1,2012-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
2,2012-12-06,Holiday,Local,Quito,Fundacion de Quito,False
3,2013-07-03,Holiday,Local,Santo Domingo,Fundacion de Santo Domingo,False
4,2013-12-05,Additional,Local,Quito,Fundacion de Quito-1,False


In [ ]:
transactions.head()

,date,store_nbr,transactions
0,2014-01-02,1,1875
1,2014-01-02,2,2122
2,2014-01-02,3,3350
3,2014-01-02,4,1661
4,2014-01-02,5,1807


In [ ]:
# Realizando o join entre as tabelas (juntar tudo em um único dataset)

# Dataset de treino
df_aux1 = pd.merge(train, oil, on='date', how='left')
df_aux2 = pd.merge(df_aux1, stores, on='store_nbr', how='left')
df_aux3 = pd.merge(df_aux2, transactions, on=['date','store_nbr'], how='left')
df_train_full = pd.merge(df_aux3, holidays_events, left_on=['date','city'], right_on=['date','locale_name'], how='left')

# Dataset de teste
df_aux1 = pd.merge(test, oil, on='date', how='left')
df_aux2 = pd.merge(df_aux1, stores, on='store_nbr', how='left')
df_aux3 = pd.merge(df_aux2, transactions, on=['date','store_nbr'], how='left')
df_test_full = pd.merge(df_aux3, holidays_events, left_on=['date','city'], right_on=['date','locale_name'], how='left')

In [ ]:
# Visualizar as 5 primeiras linhas do dataset de treino
df_train_full.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,transactions,type_holiday,locale,locale_name,description,transferred
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,0,95.14,Quito,Pichincha,D,13,1875.0,NaN,NaN,NaN,NaN,NaN
1,650431,2014-01-02,1,BABY CARE,0.0,0,95.14,Quito,Pichincha,D,13,1875.0,NaN,NaN,NaN,NaN,NaN
2,650432,2014-01-02,1,BEAUTY,6.0,0,95.14,Quito,Pichincha,D,13,1875.0,NaN,NaN,NaN,NaN,NaN
3,650433,2014-01-02,1,BEVERAGES,2524.0,0,95.14,Quito,Pichincha,D,13,1875.0,NaN,NaN,NaN,NaN,NaN
4,650434,2014-01-02,1,BOOKS,0.0,0,95.14,Quito,Pichincha,D,13,1875.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Visualizar as 5 primeiras linhas do dataset de teste
df_test_full.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,transactions,type_holiday,locale,locale_name,description,transferred
0,2867238,2017-06-02,1,AUTOMOTIVE,19.0,0,47.68,Quito,Pichincha,D,13,1925,NaN,NaN,NaN,NaN,NaN
1,2867239,2017-06-02,1,BABY CARE,0.0,0,47.68,Quito,Pichincha,D,13,1925,NaN,NaN,NaN,NaN,NaN
2,2867240,2017-06-02,1,BEAUTY,1.0,0,47.68,Quito,Pichincha,D,13,1925,NaN,NaN,NaN,NaN,NaN
3,2867241,2017-06-02,1,BEVERAGES,2752.0,27,47.68,Quito,Pichincha,D,13,1925,NaN,NaN,NaN,NaN,NaN
4,2867242,2017-06-02,1,BOOKS,0.0,0,47.68,Quito,Pichincha,D,13,1925,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Metadados referente ao conjunto de dados
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

pod_academy_generate_metadata(df_train_full)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,transferred,object,204204,99.57,1
1,description,object,204204,99.57,3
2,locale_name,object,204204,99.57,2
3,locale,object,204204,99.57,1
4,type_holiday,object,204204,99.57,2
5,oil_price,float64,63360,30.89,751
6,transactions,float64,924,0.45,2075
7,type,object,0,0.00,1
8,cluster,int64,0,0.00,4
9,id,int64,0,0.00,205095


In [ ]:
# Convertendo a coluna date para datetime
df_train_full['date'] = pd.to_datetime(df_train_full['date'])
df_test_full['date'] = pd.to_datetime(df_test_full['date'])

## 1.2 Feature Engineering

### 1.2.1 Forma "Manual"

In [ ]:
# Criando variáveis de lag
df_train_full['sales_lag_1'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(1)
df_train_full['sales_lag_3'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(3)
df_train_full['sales_lag_5'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(5)
df_train_full['sales_lag_7'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(7)
df_train_full['sales_lag_14'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(14)
df_train_full['sales_lag_18'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(18)
df_train_full['sales_lag_21'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(21)
df_train_full['sales_lag_24'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(24)
df_train_full['sales_lag_27'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(27)
df_train_full['sales_lag_30'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(30)
df_train_full['sales_lag_45'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(45)
df_train_full['sales_lag_60'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(60)
df_train_full['sales_lag_90'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(90)
df_train_full['sales_lag_120'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(120)
df_train_full['sales_lag_150'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(150)
df_train_full['sales_lag_180'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(180)
df_train_full['sales_lag_360'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].shift(360)

# Criando variáveis de médias móveis
df_train_full['sales_rolling_mean_3'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=3).mean())
df_train_full['sales_rolling_mean_5'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=5).mean())
df_train_full['sales_rolling_mean_7'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7).mean())
df_train_full['sales_rolling_mean_14'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=14).mean())
df_train_full['sales_rolling_mean_21'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=21).mean())
df_train_full['sales_rolling_mean_28'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=28).mean())
df_train_full['sales_rolling_mean_30'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=30).mean())
df_train_full['sales_rolling_mean_45'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=45).mean())
df_train_full['sales_rolling_mean_60'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=60).mean())
df_train_full['sales_rolling_mean_90'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=90).mean())
df_train_full['sales_rolling_mean_120'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=120).mean())
df_train_full['sales_rolling_mean_150'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=150).mean())
df_train_full['sales_rolling_mean_180'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=180).mean())
df_train_full['sales_rolling_mean_360'] = df_train_full.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=360).mean())

Demonstração do processo para obtenção das lags features

In [ ]:
df_aux = df_train_full[['date', 'store_nbr', 'family', 'sales', 'sales_rolling_mean_3']].copy()
df_aux[(df_aux.store_nbr == 1) & (df_aux.family == 'AUTOMOTIVE')]

,date,store_nbr,family,sales,sales_rolling_mean_3
0,2014-01-02,1,AUTOMOTIVE,3.0,NaN
165,2014-01-03,1,AUTOMOTIVE,0.0,NaN
330,2014-01-04,1,AUTOMOTIVE,0.0,1.000000
495,2014-01-05,1,AUTOMOTIVE,1.0,0.333333
660,2014-01-06,1,AUTOMOTIVE,6.0,2.333333
...,...,...,...,...,...
204270,2017-05-27,1,AUTOMOTIVE,1.0,1.333333
204435,2017-05-28,1,AUTOMOTIVE,2.0,1.333333
204600,2017-05-29,1,AUTOMOTIVE,6.0,3.000000
204765,2017-05-30,1,AUTOMOTIVE,2.0,3.333333


In [ ]:
df_train_full[['date', 'store_nbr', 'family', 'sales', 'sales_lag_3']][:5]

,date,store_nbr,family,sales,sales_lag_3
0,2014-01-02,1,AUTOMOTIVE,3.0,NaN
1,2014-01-02,1,BABY CARE,0.0,NaN
2,2014-01-02,1,BEAUTY,6.0,NaN
3,2014-01-02,1,BEVERAGES,2524.0,NaN
4,2014-01-02,1,BOOKS,0.0,NaN


Demonstração do cálculo de médias móveis com janela igual a 3

In [ ]:
df_train_full[['date', 'store_nbr', 'family', 'sales', 'sales_rolling_mean_3']][:10]

,date,store_nbr,family,sales,sales_rolling_mean_3
0,2014-01-02,1,AUTOMOTIVE,3.000,NaN
1,2014-01-02,1,BABY CARE,0.000,NaN
2,2014-01-02,1,BEAUTY,6.000,NaN
3,2014-01-02,1,BEVERAGES,2524.000,NaN
4,2014-01-02,1,BOOKS,0.000,NaN
5,2014-01-02,1,BREAD/BAKERY,388.988,NaN
6,2014-01-02,1,CELEBRATION,12.000,NaN
7,2014-01-02,1,CLEANING,852.000,NaN
8,2014-01-02,1,DAIRY,772.000,NaN
9,2014-01-02,1,DELI,141.484,NaN


### 1.2.2 Forma Automatizada

In [ ]:
def add_sales_lags(df, group_cols, lags):
    """
    Adds lagged sales columns to the DataFrame for the specified lags.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the sales data.
    group_cols (list of str): The columns to group by.
    lags (list of int): The list of lag periods to create lagged columns.

    Returns:
    pd.DataFrame: The DataFrame with added lagged sales columns.
    """
    for lag in lags:
        lag_col_name = f'sales_lag_{lag}'
        df[lag_col_name] = df.groupby(group_cols)['sales'].shift(lag)
    return df

# Exemplo de uso
group_cols = ['store_nbr', 'family']
lags = [1, 3, 5, 7, 14, 18, 21, 24, 27, 30, 45, 60, 90, 120, 150, 180, 360]
df_train_full = add_sales_lags(df_train_full, group_cols, lags)
df_test_full = add_sales_lags(df_test_full, group_cols, lags)

In [ ]:
def add_sales_rolling_means(df, group_cols, windows):
    """
    Adds rolling mean sales columns to the DataFrame for the specified window sizes.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the sales data.
    group_cols (list of str): The columns to group by.
    windows (list of int): The list of window sizes to create rolling mean columns.

    Returns:
    pd.DataFrame: The DataFrame with added rolling mean sales columns.
    """
    for window in windows:
        rolling_col_name = f'sales_rolling_mean_{window}'
        df[rolling_col_name] = df.groupby(group_cols)['sales'].transform(lambda x: x.rolling(window=window).mean())
    return df

# Exemplo de uso
group_cols = ['store_nbr', 'family']
windows = [3, 5, 7, 14, 21, 28, 30, 45, 60, 90, 120, 150, 180, 360]
df_train_full = add_sales_rolling_means(df_train_full, group_cols, windows)
df_test_full = add_sales_rolling_means(df_test_full, group_cols, windows)

### 1.2.3 Extraindo características temporais

In [ ]:
def add_date_features(df, date_col):
    """
    Adds year, month, day, day of week, and is_weekend columns to the DataFrame based on the specified date column.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the date column.
    date_col (str): The name of the date column.

    Returns:
    pd.DataFrame: The DataFrame with added date feature columns.
    """
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['is_weekend'] = df[date_col].dt.dayofweek >= 5
    return df

df_train_full = add_date_features(df_train_full, 'date')
df_test_full = add_date_features(df_test_full, 'date')

### 1.2.4 Interações entre variáveis

In [ ]:
def add_sales_oil_price_interactions(df, sales_col, oil_price_col):
    """
    Adds interaction columns between sales and oil price to the DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the sales and oil price columns.
    sales_col (str): The name of the sales column.
    oil_price_col (str): The name of the oil price column.

    Returns:
    pd.DataFrame: The DataFrame with added interaction columns.
    """
    df['sales_oil_price_interaction_mult'] = df[sales_col] * df[oil_price_col]
    df['sales_oil_price_interaction_div'] = df[sales_col] / df[oil_price_col]
    return df

df_train_full = add_sales_oil_price_interactions(df_train_full, 'sales', 'oil_price')
df_test_full = add_sales_oil_price_interactions(df_test_full, 'sales', 'oil_price')

## 1.3 Determinar Target

O objetivo do modelo preditivo será realizar previsões acuradas da demanda nos próximos 15 dias.

In [ ]:
df_train_full = df_train_full.sort_values(by=['store_nbr', 'family', 'date'])
df_train_full.loc[:,'sales_next_15_days_include_t0'] = df_train_full.groupby(['store_nbr', 'family'])['sales'] \
                                 .transform(lambda x: x.rolling(window=15, min_periods=1).sum().shift(-14))

In [ ]:
# Notar que na contagem da quantidade vendida nos próximos 15 dias é incluido o dia atual, iremos reajustar para gerar a variável target
df_train_full.head(20)[['id','date','store_nbr','family','sales','sales_next_15_days_include_t0']]

,id,date,store_nbr,family,sales,sales_next_15_days_include_t0
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,32.0
165,652212,2014-01-03,1,AUTOMOTIVE,0.0,31.0
330,653994,2014-01-04,1,AUTOMOTIVE,0.0,35.0
495,655776,2014-01-05,1,AUTOMOTIVE,1.0,35.0
660,657558,2014-01-06,1,AUTOMOTIVE,6.0,36.0
825,659340,2014-01-07,1,AUTOMOTIVE,2.0,39.0
990,661122,2014-01-08,1,AUTOMOTIVE,0.0,42.0
1155,662904,2014-01-09,1,AUTOMOTIVE,2.0,42.0
1320,664686,2014-01-10,1,AUTOMOTIVE,2.0,44.0
1485,666468,2014-01-11,1,AUTOMOTIVE,4.0,47.0


In [ ]:
df_train_full.head(15)[['id','date','store_nbr','family','sales','sales_next_15_days_include_t0']]

,id,date,store_nbr,family,sales,sales_next_15_days_include_t0
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,32.0
165,652212,2014-01-03,1,AUTOMOTIVE,0.0,31.0
330,653994,2014-01-04,1,AUTOMOTIVE,0.0,35.0
495,655776,2014-01-05,1,AUTOMOTIVE,1.0,35.0
660,657558,2014-01-06,1,AUTOMOTIVE,6.0,36.0
825,659340,2014-01-07,1,AUTOMOTIVE,2.0,39.0
990,661122,2014-01-08,1,AUTOMOTIVE,0.0,42.0
1155,662904,2014-01-09,1,AUTOMOTIVE,2.0,42.0
1320,664686,2014-01-10,1,AUTOMOTIVE,2.0,44.0
1485,666468,2014-01-11,1,AUTOMOTIVE,4.0,47.0


In [ ]:
# O target passará a indicar apenas as vendas nos próximos 15 dias (sem incluir o dia atual)
df_train_full['target'] = df_train_full['sales_next_15_days_include_t0'] - df_train_full['sales']

# Exclusão da variável sales_next_15_days
df_train_full = df_train_full.drop(columns='sales_next_15_days_include_t0')

# Retirar últimas linhas do dataset (não possuem dados de vendas de 15 dias à frente para que possam ter o target calculado)
df_train_full = df_train_full.dropna(subset='target')

## 1.4 Split do conjunto de dados

In [ ]:
# Definindo a data de corte para o split
cutoff_date = '2017-04-01'

# Separando os dados de treino e teste com base na data de corte
train_data = df_train_full[df_train_full['date'] < cutoff_date]
val_data = df_train_full[df_train_full['date'] >= cutoff_date]

# Exibindo o tamanho dos conjuntos de treino e teste
print(f"Tamanho do conjunto de treino: {train_data.shape}. Data de início: {train_data.date.min()} - Data de fim: {train_data.date.max()}")
print(f"Tamanho do conjunto de validação: {val_data.shape}. Data de início: {val_data.date.min()} - Data de fim: {val_data.date.max()}")

# Verificar tamanho do conjunto de dados de teste
print(f"Tamanho do conjunto de teste: {df_test_full.shape}. Data de início: {df_test_full.date.min()} - Data de fim: {df_test_full.date.max()}")

Tamanho do conjunto de treino: (195030, 56). Data de início: 2014-01-02 00:00:00 - Data de fim: 2017-03-31 00:00:00
Tamanho do conjunto de validação: (7755, 56). Data de início: 2017-04-01 00:00:00 - Data de fim: 2017-05-17 00:00:00
Tamanho do conjunto de teste: (2310, 55). Data de início: 2017-06-02 00:00:00 - Data de fim: 2017-06-15 00:00:00


## 1.5 Tratamento de Missings

In [ ]:
# Gerar metadados do novo dataset
pod_academy_generate_metadata(train_data)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,locale_name,object,194139,99.54,2
1,transferred,object,194139,99.54,1
2,description,object,194139,99.54,3
3,type_holiday,object,194139,99.54,2
4,locale,object,194139,99.54,1
5,sales_oil_price_interaction_div,float64,60060,30.80,81516
6,sales_oil_price_interaction_mult,float64,60060,30.80,81462
7,oil_price,float64,60060,30.80,722
8,sales_lag_360,float64,59400,30.46,34133
9,sales_rolling_mean_360,float64,59235,30.37,76502


As variáveis com menor preenchimento: description, type_holyday, transferred, locale_name, locale pertencem à base de feriados e, portanto, a ausência de preenchimento indica que se trata de um "dia normal" em que não houve feriado.

### 1.5.1 Tratando variáveis que vieram da base de feriados

* Tratando a variável type_holiday

In [ ]:
# Verificando os valores únicos
train_data.type_holiday.unique()

array([nan, 'Additional', 'Holiday'], dtype=object)

In [ ]:
# Preenchendo os campos em que os valores estão como nan
train_data.loc[:,'type_holiday'] = train_data['type_holiday'].fillna('Normal Day')
val_data.loc[:,'type_holiday'] = val_data['type_holiday'].fillna('Normal Day')

train_data.type_holiday.unique()

array(['Normal Day', 'Additional', 'Holiday'], dtype=object)

* Tratando a variável description

In [ ]:
# Verificando os valores únicos
train_data.description.unique()

array([nan, 'Fundacion de Quito-1', 'Fundacion de Quito',
       'Fundacion de Santo Domingo'], dtype=object)

In [ ]:
# Preenchendo os campos em que os valores estão como nan
train_data.loc[:,'description'] = train_data['description'].fillna('Normal Day')
val_data.loc[:,'description'] = val_data['description'].fillna('Normal Day')

train_data.description.unique()

array(['Normal Day', 'Fundacion de Quito-1', 'Fundacion de Quito',
       'Fundacion de Santo Domingo'], dtype=object)

* Tratando a variável transferred

In [ ]:
# Verificando os valores únicos
train_data.transferred.unique()

array(['Normal Day', False], dtype=object)

In [ ]:
# Preenchendo os campos em que os valores estão como nan
train_data.loc[:,'transferred'] = train_data['transferred'].fillna('Normal Day')
val_data.loc[:,'transferred'] = val_data['transferred'].fillna('Normal Day')

train_data.transferred.unique()

array(['Normal Day', False], dtype=object)

* Tratando a variável locale_name

In [ ]:
# Verificando os valores únicos
train_data.locale_name.unique()

array([nan, 'Quito', 'Santo Domingo'], dtype=object)

In [ ]:
# A variável locale_name é equivalente à variável city
train_data.city.unique()

array(['Quito', 'Santo Domingo'], dtype=object)

In [ ]:
# Remoção da variável locale_name
train_data = train_data.drop(columns='locale_name')
val_data = val_data.drop(columns='locale_name')

* Tratando a variável locale

In [ ]:
# Verificando os valores únicos
train_data.locale.unique()

array([nan, 'Local'], dtype=object)

In [ ]:
# Preenchendo os campos em que os valores estão como nan
train_data.loc[:,'locale'] = train_data['locale'].fillna('Normal Day')
val_data.loc[:,'locale'] = val_data['locale'].fillna('Normal Day')

train_data.locale.unique()

array(['Normal Day', 'Local'], dtype=object)

### 1.5.2 Tratando variáveis que vieram da base de preços de petróleo

* Tratando variável oil_price

In [ ]:
train_data[np.isnan(train_data.oil_price)].is_weekend.value_counts(normalize=True).reset_index()

,is_weekend,proportion
0,True,0.925824
1,False,0.074176


92,5% dos campos missing nessa variável ocorrem aos finais de semana.

In [ ]:
train_data[np.isnan(train_data.oil_price)].head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
330,653994,2014-01-04,1,AUTOMOTIVE,0.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,4,5,True,NaN,NaN,35.0
495,655776,2014-01-05,1,AUTOMOTIVE,1.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,5,6,True,NaN,NaN,34.0
1485,666468,2014-01-11,1,AUTOMOTIVE,4.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,11,5,True,NaN,NaN,43.0
1650,668250,2014-01-12,1,AUTOMOTIVE,2.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,12,6,True,NaN,NaN,42.0
2640,678942,2014-01-18,1,AUTOMOTIVE,4.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,18,5,True,NaN,NaN,39.0


In [ ]:
train_data.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,...,sales_rolling_mean_180,sales_rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,0,95.14,Quito,Pichincha,D,...,NaN,NaN,2014,1,2,3,False,285.42,0.031532,29.0
165,652212,2014-01-03,1,AUTOMOTIVE,0.0,0,93.66,Quito,Pichincha,D,...,NaN,NaN,2014,1,3,4,False,0.00,0.000000,31.0
330,653994,2014-01-04,1,AUTOMOTIVE,0.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,4,5,True,NaN,NaN,35.0
495,655776,2014-01-05,1,AUTOMOTIVE,1.0,0,NaN,Quito,Pichincha,D,...,NaN,NaN,2014,1,5,6,True,NaN,NaN,34.0
660,657558,2014-01-06,1,AUTOMOTIVE,6.0,0,93.12,Quito,Pichincha,D,...,NaN,NaN,2014,1,6,0,False,558.72,0.064433,30.0


In [ ]:
# Preencher com o último valor conhecido (dia anterior ao missing)
train_data['oil_price'] = train_data['oil_price'].bfill()
val_data['oil_price'] = val_data['oil_price'].bfill()

train_data['sales_oil_price_interaction_div'] = train_data['sales_oil_price_interaction_div'].bfill()
val_data['sales_oil_price_interaction_div'] = val_data['sales_oil_price_interaction_div'].bfill()

train_data['sales_oil_price_interaction_mult'] = train_data['sales_oil_price_interaction_mult'].bfill()
val_data['sales_oil_price_interaction_mult'] = val_data['sales_oil_price_interaction_mult'].bfill()

In [ ]:
# Verificando os missings restantes
pod_academy_generate_metadata(train_data)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,sales_lag_360,float64,59400,30.46,34133
1,sales_rolling_mean_360,float64,59235,30.37,76502
2,sales_lag_180,float64,29700,15.23,40914
3,sales_rolling_mean_180,float64,29535,15.14,84038
4,sales_lag_150,float64,24750,12.69,42053
5,sales_rolling_mean_150,float64,24585,12.61,84637
6,sales_lag_120,float64,19800,10.15,43230
7,sales_rolling_mean_120,float64,19635,10.07,84297
8,sales_lag_90,float64,14850,7.61,44453
9,sales_rolling_mean_90,float64,14685,7.53,83283


* Tratando os missings oriundos do processo de feature engineering

In [ ]:
# Remover missings na base de treino
df_train_new = train_data.dropna()

# Remover missings na base de validação
df_val_new = val_data.dropna()

In [ ]:
vars_train_001 = vars_train_001 + ['target']

# Salvando a versão simplificada - apenas as variáveis que estavam na base train original
df_train_new[vars_train_001].to_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/train_001.csv', index=False)
df_val_new[vars_train_001].to_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/val_001.csv', index=False)

# Salvando a versão completa - contendo todas as variáveis geradas neste notebook
df_train_new.to_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/train_002.csv', index=False)
df_val_new.to_csv('/content/drive/MyDrive/ProjetosDataScience/Previsao_Demandas/val_002.csv', index=False)